In [1]:
import polars as pl

In [ ]:
def normalize(tag: str) -> str:
    parts = tag.split('/')
    return parts

In [ ]:
tickets = pl.read_csv("tickets.csv")

In [ ]:
tickets.with_columns(pl.select(pl.col('tags')).str.split(",").alias("tags_list"))

In [ ]:
tickets.columns

In [ ]:
def tags_listed(alias: str):
    strip_lower = pl.element().str.strip_chars(" ").str.to_lowercase().str.replace(r"\s", "-")
    foreach_tag = pl.element().str.split("/").list.eval(strip_lower).list.join("/")
    select_tags = pl.col("tags").str.split(",").list.eval(foreach_tag).alias(alias)
    return tickets.select(select_tags)

# tickets.select(pl.col("tags").str.split(",").list.eval(pl.element().str.strip_chars(" ").str.to_lowercase()))
# tickets.with_columns(tickets.select(
#     pl.col("tags").str.split(",").list.eval(strip_lower)
# ))

tickets = tickets.with_columns(tags_listed("tags_list"))

In [ ]:
tickets.select(pl.col("ticket_id"), pl.col("tags_list")).explode("tags_list")

In [ ]:
tags = tickets.explode("tags_list").select(pl.col("tags_list").alias("tag"), pl.

tags

In [2]:
import load_data

In [ ]:
def ticks_list():
    strip_lower = pl.element().str.strip_chars(" ").str.to_lowercase().str.replace(r"\s", "-")
    foreach_tag = pl.element().str.split("/").list.eval(strip_lower).list.join("/")
    select_tags = pl.col("tags").str.split(",").list.eval(foreach_tag)
    return select_tags

def clean_tags():
    normalize = pl.element().str.strip_chars(" ").str.to_lowercase().str.replace(r"\s", "-")
    clean_tag = pl.element().str.split("/").list.eval(normalize).list.join("/")
    q = pl.col("tags").str.split(",").list.eval(clean_tag)
    return q


In [3]:
tickets = load_data.get_tickets()
# tickets = pl.read_csv("data/tickets.csv")
tickets

ticket_id,subject,body_text,tags
i64,str,str,list[str]
348342800,"""Shipment""","""Hello, I’m currently out of t…","[""general-product/safety"", ""complaint/heat""]"
348344318,"""issue with order -ORDER NO. #6…","""hi there! i placed an order an…","[""urgent"", ""internal/log"", … ""wcon-p""]"
343560310,"""Re: Order #579121 confirmed""","""Hello, I still haven’t receiv…","[""complaint/smell-taste"", ""complaint-date/2024-08-17"", … ""order/promo""]"
348376971,"""Fwd: A shipment from order #61…","""Hello, Please cancel this shi…","[""urgent"", ""refund/denied"", … ""wtr1-p""]"
348388616,"""Issue with pack""","""Hi, I ordered the first trim…","[""internal/log"", ""complaint/smell-taste"", … ""other/fishy""]"
…,…,…,…
368741112,"""786242 damaged product""",""" Sent from my iPhone""","[""complaint/damaged"", ""email""]"
368742742,"""Cancel subscription""","""Hi, I’d like to cancel my sub…","[""siena/stop-response"", ""cancel/no-reason"", … ""complaint-date/2024-12-31""]"
368762239,"""Return item""","""Return item -----------------…","[""#675926"", ""complaint/side-effects"", … ""offline-capture""]"


In [9]:
tickets.explode("tags").group_by(pl.col("tags").alias("tag")).agg(
    pl.len().alias("count"),
    pl.col("ticket_id").alias("ticket_ids"),
).sort("count", descending=True)

tag,count,ticket_ids
str,u32,list[i64]
"""internal/log""",1556,"[348344318, 343560310, … 368785518]"
"""cancel/do-not-like""",753,"[343560310, 348405986, … 368766370]"
"""refund/30-day""",752,"[343560310, 348405986, … 368766370]"
"""complaint/smell-taste""",715,"[343560310, 348388616, … 368742742]"
"""wtr1-p""",625,"[343560310, 348376971, … 368742742]"
…,…,…
"""#620217""",1,[358387695]
"""#739333""",1,[362368791]
"""#680141""",1,[356651966]


In [17]:
df = tickets.select(pl.col("ticket_id"), pl.col("body_text"), pl.col("tags")).to_pandas()

In [19]:
df[:5]

,ticket_id,body_text,tags
0,348342800,"Hello,\n\nI’m currently out of the country wil...","[general-product/safety, complaint/heat]"
1,348344318,hi there! i placed an order and only part of t...,"[urgent, internal/log, complaint/quality, repl..."
2,343560310,"Hello, \nI still haven’t received my refund. A...","[complaint/smell-taste, complaint-date/2024-08..."
3,348376971,"Hello, \nPlease cancel this shipment and send ...","[urgent, refund/denied, no-csat, refund/courte..."
4,348388616,"Hi, \n\nI ordered the first trimester pack and...","[internal/log, complaint/smell-taste, wtr1-p, ..."


In [26]:
for i, row in df[:10].iterrows():
    print(f"index: {i}")
    print(f"ticket: {row['ticket_id']}")

index: 0
ticket: 348342800
index: 1
ticket: 348344318
index: 2
ticket: 343560310
index: 3
ticket: 348376971
index: 4
ticket: 348388616
index: 5
ticket: 348395199
index: 6
ticket: 348405986
index: 7
ticket: 348414135
index: 8
ticket: 348414173
index: 9
ticket: 348089071


In [23]:
df[0]

KeyError: 0

In [24]:
df[1]

KeyError: 1